# Exploração de Tweets


## CARREGA DADOS E PACOTES

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import networkx as nx 
from networkx.algorithms.community import k_clique_communities, girvan_newman, greedy_modularity_communities

import re
from collections import Counter
import itertools

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import decomposition

from nltk.corpus import stopwords
import wordcloud
from efficient_apriori import apriori

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
import hdbscan
import scipy.cluster.hierarchy as shc

import spacy
import scattertext as st

import json
import nltk

from gensim.corpora import Dictionary
import pyLDAvis.gensim
import gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

from IPython.display import HTML

from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

import utils

import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "on

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))
plt.rcParams["figure.figsize"] = (60,30)
plt.rcParams['figure.dpi'] = 90
plt.rcParams.update({'font.size': 40})

In [ ]:
Mystopwords = utils.mystopwords

In [ ]:
df = pd.read_pickle('dados\\df_processado.pkl')
df.info()

In [ ]:
print("Total de Tweets:", df[df['data']>"2020-03-01"].shape[0])
print("Total de Usuários:", df['usuario'].unique().shape[0])

Seleciona usuários com mais de 10 tweets publicados

In [ ]:
df = df[df['data']>"2020-03-01"]
df = df[~df['sent_manual'].isin(['D'])] #remove tweets marcadaos com D (delete)
df = df[~df['usuario'].isin(['BancoCentralBR'])] #remove BancoCentralBR

Nr_min_tweets = 10

lista_usuarios = df['usuario'].value_counts()[df['usuario'].value_counts()>Nr_min_tweets].index.values.tolist()
print("Número de usuários com mais de", Nr_min_tweets,"Tweets:",len(lista_usuarios))
df = df[df['usuario'].isin(lista_usuarios)]


lista_usuarios2 = lista_usuarios

df.info()

In [ ]:
# REMOVE TWEETS_LIMPOS SEM TOKENS 
df=df[df['tweet_limpo'].str.len()>0]
# REMOVE TWEETS COM POUCAS PALAVRAS
df = df[df['tweet_limpo'].str.split().apply(lambda x: len(x))>10]

In [ ]:
print("Total de Tweets:", df[df['data']>"2020-03-01"].shape[0])
print("Total de Usuários:", df['usuario'].unique().shape[0])

Seleciona a maior comunidade da rede de menções e retweets

df = df[df['usuario'].isin(lista_usuarios2)]
df.info()

In [ ]:
print("Total de Tweets:", df[df['data']>"2020-03-01"].shape[0])
print("Total de Usuários:", df['usuario'].unique().shape[0])

In [ ]:
df['data2'] = df['data'].apply(lambda x: x.date())
df['data2'].min()

In [ ]:
df['data2'].max()

In [ ]:
print("Número de dias:", (np.max(df['data2'])-np.min(df['data2'])).days)
print("Número médio de tweets por dia:", df.shape[0]/(np.max(df['data2'])-np.min(df['data2'])).days)
print("Número de semanas:", (np.max(df['data2'])-np.min(df['data2'])).days//7)

# 1. ANÁLISE DESCRITIVA 

## 1.1 MÉTRICAS DE TWEETS 

## Nr de tweets

### por dia 

In [ ]:
print('média: ', df['data'].apply(lambda x: x.date()).value_counts().mean())
print('desvio padrão: ', df['data'].apply(lambda x: x.date()).value_counts().std())

In [ ]:
df['data'].apply(lambda x: x.date()).value_counts().sort_index().asfreq('D').fillna(0).plot(linewidth = 8, title = "nr de tweets por dia")

#### Histograma das publicações diárias

In [ ]:
df['data'].apply(lambda x: x.date()).value_counts().sort_index().asfreq('D').fillna(0).hist(bins = 20)

### por dia e mês

In [ ]:
df['day'] = df['data'].dt.day
df['month'] = df['data'].dt.month

# -------------------------- Heat map ------------------------------
df1 = df.groupby(["day", "month"]).count().reset_index()
hm = df1.pivot("day", "month", "tweet")
ax = sns.heatmap(hm, cmap=sns.cm.rocket_r)
plt.show()

In [ ]:
#https://stackoverflow.com/questions/52910187/how-to-make-a-polygon-radar-spider-chart-in-python

plt.rcParams.update({'font.size': 10})

def radar_factory(num_vars, frame='circle'):
    """Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle' | 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarAxes(PolarAxes):

        name = 'radar'

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.concatenate((x, [x[0]]))
                y = np.concatenate((y, [y[0]]))
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k")
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

        def draw(self, renderer):
            """ Draw. If frame is polygon, make gridlines polygon-shaped """
            if frame == 'polygon':
                gridlines = self.yaxis.get_gridlines()
                for gl in gridlines:
                    gl.get_path()._interpolation_steps = num_vars
            super().draw(renderer)


        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                                    + self.transAxes)


                return {'polar': spine}
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

lista_dados1 = df[df['month']==3]['data'].dt.day.value_counts().sort_index().values.tolist()
lista_dados2 = df[df['month']==4]['data'].dt.day.value_counts().sort_index().values.tolist()+['0']
lista_dados3 = df[df['month']==5]['data'].dt.day.value_counts().sort_index().values.tolist()
lista_dados4 = df[df['month']==6]['data'].dt.day.value_counts().sort_index().values.tolist()+['0']
lista_dados5 = df[df['month']==7]['data'].dt.day.value_counts().sort_index().values.tolist()
lista_dados6 = df[df['month']==8]['data'].dt.day.value_counts().sort_index().values.tolist()
lista_dados6 = lista_dados6+['0' for i in range(31-len(lista_dados6))]

lista_dados1.reverse()
lista_dados2.reverse()
lista_dados3.reverse()
lista_dados4.reverse()
lista_dados5.reverse()
lista_dados6.reverse()
lista_dias = [str(i) for i in range(1,32)]
lista_dias.reverse()

data = [lista_dias,
        ('Tweets por dia do mês', [
            lista_dados1,
            lista_dados2,
            lista_dados3,
            lista_dados4,
            lista_dados5,
            lista_dados6])]

N = len(data[0])
theta = radar_factory(N, frame='polygon')

spoke_labels = data.pop(0)
title, case_data = data[0]

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(projection='radar'))
fig.subplots_adjust(top=0.85, bottom=0.05)

ax.set_rgrids([50, 100, 150, 200])
ax.set_title(title,  position=(0.5, 1.1), ha='center')

for d in case_data:
    line = ax.plot(theta, d)
    ax.fill(theta, d,  alpha=0.25)
ax.set_varlabels(spoke_labels)

# add legend relative to top-left plot
labels = ('Março', 'Abril', 'Maio', 'Junho','julho','agosto')
plt.legend(labels, loc=(0.9, .95), labelspacing=0.1, fontsize='small')

plt.show()

In [ ]:
plt.rcParams.update({'font.size': 40})

### por dia da semana

In [ ]:
df['data'].dt.dayofweek\
.apply(lambda x: "segunda" if x==0 else "terça" if x==1 else "quarta" if x==2 else "quinta" if x==3 else "sexta" if x==4 else "sábado" if x==5 else "domingo" )\
.value_counts().reindex(["domingo","segunda","terça","quarta","quinta","sexta","sábado"]).plot.bar()

In [ ]:
#https://stackoverflow.com/questions/52910187/how-to-make-a-polygon-radar-spider-chart-in-python

plt.rcParams.update({'font.size': 10})

lista_dados = df['data'].dt.dayofweek\
.apply(lambda x: "segunda" if x==0 else "terça" if x==1 else "quarta" if x==2 else "quinta" if x==3 else "sexta" if x==4 else "sábado" if x==5 else "domingo" )\
.value_counts().reindex(["domingo","segunda","terça","quarta","quinta","sexta","sábado"]).values.tolist()

lista_dados.reverse()
lista_dias_semana = ['Domingo', 'Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado']
lista_dias_semana.reverse()

data = [lista_dias_semana,
        ('Tweets por dia da semana', [
            lista_dados])]

N = len(data[0])
theta = radar_factory(N, frame='polygon')

spoke_labels = data.pop(0)
title, case_data = data[0]

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(projection='radar'))
fig.subplots_adjust(top=0.85, bottom=0.05)

ax.set_rgrids([200, 400, 600, 800])
ax.set_title(title,  position=(0.5, 1.1), ha='center')

for d in case_data:
    line = ax.plot(theta, d)
    ax.fill(theta, d,  alpha=0.25)
ax.set_varlabels(spoke_labels)

plt.show()

In [ ]:
plt.rcParams.update({'font.size': 40})

### acumulado por hora do dia 

In [ ]:
#df['data'].apply(lambda x: x.hour)
df['data'].dt.hour.value_counts().sort_index().plot(linewidth = 10, title = "nr de tweets acumulados por hora do dia")

In [ ]:
#https://stackoverflow.com/questions/52910187/how-to-make-a-polygon-radar-spider-chart-in-python

plt.rcParams.update({'font.size': 10})

lista_dados = df['data'].dt.hour.value_counts().sort_index().values.tolist()

lista_dados.reverse()
lista_horas = ['0h','1h','2h','3h','4h','5h','6h','7h','8h','9h','10h','11h','12h','13h','14h','15h','16h','17h','18h','19h','20h','21h','22h','23h']
lista_horas.reverse()

data = [lista_horas,
        ('Tweets por hora do dia', [
            lista_dados])]

N = len(data[0])
theta = radar_factory(N, frame='polygon')

spoke_labels = data.pop(0)
title, case_data = data[0]

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(projection='radar'))
fig.subplots_adjust(top=0.85, bottom=0.05)

ax.set_rgrids([100, 200, 300, 400, 500])
ax.set_title(title,  position=(0.5, 1.1), ha='center')

for d in case_data:
    line = ax.plot(theta, d)
    ax.fill(theta, d,  alpha=0.25)
ax.set_varlabels(spoke_labels)

plt.show()

In [ ]:
plt.rcParams.update({'font.size': 40})

### por hora 

In [ ]:
# a list of "1" to count the hashtags
ones = [1]*df[df['data']>"2020-03-01"]['data'].shape[0]
# the index of the series
idx = pd.DatetimeIndex(df[df['data']>"2020-03-01"]['data'])
# Resampling / bucketing
pd.Series(ones, index=idx).resample('H').sum().fillna(0).plot(title = "Nr de tweets por hora")
 

#### Histograma das publicações por hora

In [ ]:
pd.Series(ones, index=idx).resample('H').sum().fillna(0).hist(bins = 40)

### número de likes por dia

In [ ]:
df[['data2','likes']].groupby('data2').sum().plot(xlim=('2020-03-01',np.max(df['data'])),linewidth = 10, title = "Nr likes por dia")

### número de retweets por dia

In [ ]:
df[['data2','retweets']].groupby('data2').sum().plot(xlim=('2020-03-01',np.max(df['data'])),linewidth = 10, title = "Nr retweets por dia")

### Source

In [ ]:
df['source'].value_counts().sort_values(ascending=False).head(5).plot.bar()

### Maior número de likes

In [ ]:
df.sort_values(by='likes', ascending=False)[['data2','usuario','tipo_usuario','seguidores','likes','retweets','tweet']].head(5)

### Maior número de retweets

In [ ]:
df.sort_values(by='retweets', ascending=False)[['data2','usuario','tipo_usuario','seguidores','likes','retweets','tweet']].head(5)

### Maior número de respostas

In [ ]:
if df['replies'].max()>0:
    df.sort_values(by='replies', ascending=False)[['data2','usuario','tipo_usuario','seguidores','likes','retweets','replies','tweet']].head(5)
else:
    print("Não há tweets com replies > 0!")

### comprimento em caracteres do tweet

In [ ]:
print("Comprimento médio: ", df['tweet'].str.len().mean())

In [ ]:
df['tweet'].str.len().hist()

### número de palavras por tweet

In [ ]:
print("Número médio de palavras por tweet", df['tweet'].str.split().map(lambda x: len(x)).mean())

In [ ]:
df['tweet'].str.split().\
    map(lambda x: len(x)).\
    hist()

### Número de palavras publicadas por dia 

In [ ]:
df['nr_palavras'] = df['tweet_limpo'].apply(lambda x: len(x.split(" ")))
df[['data2','nr_palavras']].groupby('data2').sum().plot(xlim=('2020-03-01',np.max(df['data'])),linewidth = 10, title = "Nr palavras publicadas por dia")

## 1.2 MÉTRICAS DE USUÁRIOS 

### total de usuários

In [ ]:
df['usuario'].unique().shape[0]

In [ ]:
#lista_usuarios = df['usuario'].unique().tolist()
lista_usuarios

### usuários ativos por dia 

In [ ]:
df[['data2','usuario']].drop_duplicates()['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])),linewidth = 10,title = "Número de usuários ativos por dia")

### tweets por usuário

In [ ]:
df['usuario'].value_counts().sort_values(ascending=False).head(20).plot.bar(title = "nr de tweets por usuários")

### Usuários com mais retweets

In [ ]:
df[['usuario','retweets']].groupby('usuario').sum().sort_values(by = "retweets",ascending=False).head(20).plot.bar(title = "Retweets por usuário")

### Usuários com maiores médias de retweets

In [ ]:
df[['usuario','retweets']].groupby('usuario').mean().sort_values(by = "retweets",ascending=False).head(20).plot.bar(title = "Média de Retweets por usuário")

### Usuários com mais likes

In [ ]:
df[['usuario','likes']].groupby('usuario').sum().sort_values(by = "likes",ascending=False).head(20).plot.bar(title = "Likes por usuário")

### Usuários com maiores médias de likes

In [ ]:
df[['usuario','likes']].groupby('usuario').mean().sort_values(by = "likes",ascending=False).head(20).plot.bar(title = "Média de Likes por usuário")

### usuários com mais seguidores

In [ ]:
df[['usuario','seguidores']].groupby('usuario').max().sort_values(by = 'seguidores',ascending=False).head(20).plot.bar(title = "nr de seguidores por usuário")

### usuários com mais amigos

In [ ]:
df[['usuario','amigos']].groupby('usuario').max().sort_values(by = 'amigos',ascending=False).head(20).plot.bar(title = "nr de amigos por usuário")

### Follow ratio

Razão entre o número de seguidores e o de seguidos

In [ ]:
df['follow_ratio'] = df['seguidores']/df['amigos'] #risco de dar divisão por zero
df['follow_ratio']=df['follow_ratio'].apply(lambda x: np.nan if x==np.inf else x) #remove os infinitos gerados pela divisão por zero
df[['usuario','follow_ratio']].groupby('usuario').max().sort_values(by = 'follow_ratio',ascending=False).head(20).plot.bar(title = "Follow ratio por usuário")

### Usuários com mais publicações (statuses) 

Mede a atividade do usuário na rede social. Quantos tweets o usuário publicou no total.

In [ ]:
df[['usuario','nr_tweets']].groupby('usuario').max().sort_values(by = 'nr_tweets',ascending=False).head(20).plot.bar(title = "nr de tweets publicados (statuses) por usuário")

### Usuários com mais favoritos

In [ ]:
df[['usuario','favoritos']].groupby('usuario').max().sort_values(by = 'favoritos',ascending=False).head(20).plot.bar(title = "nr de tweets considerados favoritos pelo usuário")

### usuários mais citados

In [ ]:
df.tweet.str.extractall(r'(\@\w+)')[0].value_counts().head(30).plot.bar(title = "usuários mais citados nos tweets")
print("Número de usuários citados:", df.tweet.str.extractall(r'(\@\w+)')[0].value_counts().shape[0])

### usuários da base mais citados

In [ ]:
s = df.tweet.str.extractall(r'(\@\w+)')[0].value_counts()
s[s.index.str.contains('|'.join(lista_usuarios2))].head(30).plot.bar(title = "usuários economistas e jornalistas mais citados")
print("Número de usuários economistas citados:", s[s.index.str.contains('|'.join(lista_usuarios2))].shape[0])

### tipos de usuários 

#### Total 

In [ ]:
df['tipo_usuario'].value_counts().sort_values(ascending=False)

In [ ]:
df['tipo_usuario'].value_counts().sort_values(ascending=False).plot.pie()

In [ ]:
df['tipo_usuario'].value_counts().sort_values(ascending=False).plot.bar(title = "nr de usuários por tipo")

In [ ]:
df.groupby('tipo_usuario')[['seguidores','likes','retweets']].mean()

### Localidades dos usuários

In [ ]:
df['localidade'].value_counts().to_frame().head(20)

### Descrição dos usuários

#### Frequência de palavras das descrições

In [ ]:
word_count_vect3 = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', stop_words = Mystopwords+['of','rio','and','are','my','co','own','https','the','t','ex','at','in','freeassange','freepalestine','4v5qxkjosj','acompanhe','sauvyyjbfe'])
word_count_vect3.fit( [x for x in  df['description'].unique().tolist()  if x is not None]  );

In [ ]:
bag_of_words3 = word_count_vect3.transform([x for x in  df['description'].unique().tolist()  if x is not None] )
sum_words = bag_of_words3.sum(axis=0)
words_freq3 = [(word, sum_words[0, idx]) for word, idx in word_count_vect3.vocabulary_.items()]
words_freq3 =sorted(words_freq3, key = lambda x: x[1], reverse=True)

In [ ]:
number_of_words = 20
y_pos = np.arange(number_of_words)
objects = []
performance = []
for i in range(number_of_words):
    aux = words_freq3[i]
    objects.append(aux[0])
    performance.append(aux[1])
del number_of_words

In [ ]:
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Frequência')
plt.ylabel('Palavras das descrições')
plt.title('Frequência de palavras das descrições de usuários sem stop words')
plt.show()

#### Nuvem de palavras das descrições

In [ ]:
cloud = wordcloud.WordCloud(stopwords=Mystopwords+['de','da','do','em']+['of','rio','and','are','my','co','own','https','the','t','ex','at','in','freeassange','freepalestine','4v5qxkjosj','acompanhe','sauvyyjbfe','http','and'], max_font_size=50, max_words=100, background_color="white").\
generate(' '.join( [x for x in  df['description'].unique().tolist()  if x is not None] ).lower())
# Display the generated image:
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Associação entre citações de usuários

* __Support__: This says how popular an itemset is, as measured by the proportion of transactions in which an itemset appears.
* __Confidence__: This says how likely item Y is purchased when item X is purchased, expressed as {X -> Y}. This is measured by the proportion of transactions with item X, in which item Y also appears.
* __Lift__: This says how likely item Y is purchased when item X is purchased, while controlling for how popular item Y is. A lift value greater than 1 means that item Y is likely to be bought if item X is bought, while a value less than 1 means that item Y is unlikely to be bought if item X is bought.

Fonte: https://www.kdnuggets.com/2016/04/association-rules-apriori-algorithm-tutorial.html

In [ ]:
lista_de_lista_de_usuarios=[]
for tweet in df['tweet']:
    l = re.findall(r'(\@\w+)',tweet)
    if len(l)>1:
        lista_de_lista_de_usuarios.append(l)

#aplica o algoritmo apriori
itemsets, rules = apriori(lista_de_lista_de_usuarios, min_support=0.01,  min_confidence=0.4)
rules_rhs = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
    print(rule) 

### Clustering de usuários

Baseado em https://github.com/twitterdev/do_more_with_twitter_data/blob/master/examples/clustering_users/clustering-users.ipynb

In [ ]:
vec = TfidfVectorizer(preprocessor=utils.remove_url, 
                      stop_words=Mystopwords+\
                      ['of','rio','and','are','my','co','own','https','the','t','ex','at','rt','nunca','tudo',\
                      'alvaro','si'],
                      
                     )

In [ ]:
df_aux = df[['usuario','description']].copy()
df_aux = df_aux.drop_duplicates(subset = ['usuario'],keep = 'last')

unique_user_map = {}
# create one entry per user
for i in range(df_aux.shape[0]):
    unique_user_map[df_aux['usuario'].values.tolist()[i]] = df_aux['description'].values.tolist()[i].lower()


# we need to maintain the same ordering of users and bios
unique_users = []
unique_bios = []
for user,bio in unique_user_map.items():
    unique_users.append(user)
    if bio is None:
        # special case for empty bios
        bio = ''
    unique_bios.append(bio)

del df_aux

In [ ]:
bio_matrix = vec.fit_transform(unique_bios)

In [ ]:
bio_matrix.shape

In [ ]:
hdbs = hdbscan.HDBSCAN(min_cluster_size=10,
                               prediction_data=True,
                               core_dist_n_jobs=-1,
                               memory='data')
hdbs.fit(bio_matrix.todense())

In [ ]:
# get the population sizes 
label_counts = Counter(hdbs.labels_)
xs, ys = [], []
for k,v in label_counts.items():
    xs.append(k)
    ys.append(v)

# draw the chart
plt.bar(xs, ys)

plt.xticks(range(-1, len(label_counts)))
plt.ylabel('population')
plt.xlabel('cluster label')
plt.title('population sizes ({} clusters found by hdbscan)'.format(len(label_counts) - 1));

In [ ]:
def strongest_features(model, vectorizer, matrix, topk=10):
    """
    Helper function to display a simple text representation of the top-k most 
    important features in our fit model and vectorizer.
    
    matrix: matriz obtida da função vec.fit_transform
    vectorizer: sklearn vectorizer
    topk: k numbers of words to get per cluster
    
    """
    features = vectorizer.get_feature_names()
    # ignore noise labels
    relevant_labels = [ x for x in set(model.labels_) if x >= 0 ]
    for this_label in relevant_labels:
        matching_rows = np.where(hdbs.labels_ == this_label)[0]
        coeff_sums = np.sum(matrix[matching_rows], axis=0).A1
        sorted_coeff_idxs = np.argsort(coeff_sums)[::-1]
        print('Cluster {}: '.format(this_label), end='')
        for idx in sorted_coeff_idxs[:topk]:
            print('{} '.format(features[idx]), end='')
        print()
 

In [ ]:
def strongest_features_per_cluster(cluster, model, vectorizer, matrix, topk=10):
    """
    Helper function to display a simple text representation of the top-k most 
    important features in our fit model and vectorizer.
    
    cluster: cluster number
    matrix: matriz obtida da função vec.fit_transform
    vectorizer: sklearn vectorizer
    topk: k numbers of words to get per cluster
    
    """
    features = vectorizer.get_feature_names()
    # ignore noise labels
    relevant_labels = [ x for x in set(model.labels_) if x >= 0 ]
    this_label = cluster
    matching_rows = np.where(hdbs.labels_ == this_label)[0]
    coeff_sums = np.sum(matrix[matching_rows], axis=0).A1
    sorted_coeff_idxs = np.argsort(coeff_sums)[::-1]
    print('Cluster {}: '.format(this_label), end='')
    for idx in sorted_coeff_idxs[:topk]:
        print('{} '.format(features[idx]), end='')
    print()
 

In [ ]:
strongest_features(hdbs, vec, bio_matrix, topk=15)

## 1.3 MÉTRICAS DE URL 

### Possui links

In [ ]:
df['tweet'].apply(lambda x: "https://t.co" in x).value_counts().plot.bar()

### Sites mais compartilhados

In [ ]:
#carrega dicionário de urls
pickle_in = open("dados\\dict_url.pickle","rb")
dict_url = pickle.load(pickle_in)
#pega lista de urls dos tweets
lista_urls = df.tweet.str.extractall(r'(http\S+)')[0].values.tolist()
lista_real_urls = []
for shorturl in lista_urls:
    if shorturl in dict_url.keys():
        lista_real_urls.append(dict_url[shorturl])
    else:
        lista_real_urls.append("NA")
        


lista_domain = []
for url in lista_real_urls:
    if url != "NA":
        lista_domain.append(url.split("//")[-1].split("/")[0].split('?')[0])

print("Número total de urls:",len(lista_urls))
print("Número de urls válidas:",len(lista_domain),"\n")
        
domains = Counter(lista_domain)
domains.most_common()

In [ ]:
keys = []
values = []
for key, value in domains.most_common(15):
    keys.append(key)
    values.append(value)

y_pos = np.arange(len(keys))

plt.barh(y_pos, values, align='center', alpha=0.4)
plt.yticks(y_pos, keys)
plt.xlabel('Número de compartilhamentos')
plt.title('Domínio das urls mais compartilhadas')

plt.show()

# 2. ANÁLISE DE CONTEÚDO

## 2.1 ANÁLISE DE PALAVRAS 

## Word Frequency

In [ ]:
word_count_vect2 = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', stop_words = Mystopwords)
word_count_vect2.fit(df['tweet_limpo'].to_list());

In [ ]:
bag_of_words2 = word_count_vect2.transform(df['tweet_limpo'].to_list())
sum_words = bag_of_words2.sum(axis=0)
words_freq2 = [(word, sum_words[0, idx]) for word, idx in word_count_vect2.vocabulary_.items()]
words_freq2 =sorted(words_freq2, key = lambda x: x[1], reverse=True)

In [ ]:
number_of_words = 20
y_pos = np.arange(number_of_words)
objects = []
performance = []
for i in range(number_of_words):
    aux = words_freq2[i]
    objects.append(aux[0])
    performance.append(aux[1])
del number_of_words

In [ ]:
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Frequência')
plt.ylabel('Palavras')
plt.title('Frequência de palavras sem stop words')
plt.show()

## Bigram Frequency

In [ ]:
bigram_count_vect2 = CountVectorizer(analyzer='word', ngram_range=(2, 2),stop_words = Mystopwords)
bigram_count_vect2.fit(df['tweet_limpo'].to_list());

In [ ]:
bag_of_bigrams2 = bigram_count_vect2.transform(df['tweet_limpo'].to_list())
sum_bigrams = bag_of_bigrams2.sum(axis=0)
bigrams_freq2 = [(bigram, sum_bigrams[0, idx]) for bigram, idx in bigram_count_vect2.vocabulary_.items()]
bigrams_freq2 =sorted(bigrams_freq2, key = lambda x: x[1], reverse=True)

In [ ]:
number_of_bigrams = 20
y_pos = np.arange(number_of_bigrams)
objects = []
performance = []
for i in range(number_of_bigrams):
    aux = bigrams_freq2[i]
    objects.append(aux[0])
    performance.append(aux[1])
del number_of_bigrams

In [ ]:
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Frequency')
plt.ylabel('Bigrams')
plt.title('Bigrams Frequency without stop words')
plt.show()

## Trigram Frequency

In [ ]:
trigram_count_vect2 = CountVectorizer(analyzer='word', ngram_range=(3, 3),stop_words = Mystopwords)
trigram_count_vect2.fit(df['tweet_limpo'].to_list());

In [ ]:
bag_of_trigrams2 = trigram_count_vect2.transform(df['tweet_limpo'].to_list())
sum_trigrams = bag_of_trigrams2.sum(axis=0)
trigrams_freq2 = [(trigram, sum_trigrams[0, idx]) for trigram, idx in trigram_count_vect2.vocabulary_.items()]
trigrams_freq2 =sorted(trigrams_freq2, key = lambda x: x[1], reverse=True)

In [ ]:
number_of_trigrams = 20
y_pos = np.arange(number_of_trigrams)
objects = []
performance = []
for i in range(number_of_trigrams):
    aux = trigrams_freq2[i]
    objects.append(aux[0])
    performance.append(aux[1])
del number_of_trigrams

In [ ]:
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Frequency')
plt.ylabel('Trigrams')
plt.title('Trigrams Frequency without stop words')
plt.show()

## 4gram Frequency

In [ ]:
fourgram_count_vect2 = CountVectorizer(analyzer='word', ngram_range=(4, 4),stop_words = Mystopwords)
fourgram_count_vect2.fit(df['tweet_limpo'].to_list());

In [ ]:
bag_of_fourgrams2 = fourgram_count_vect2.transform(df['tweet_limpo'].to_list())
sum_fourgrams = bag_of_fourgrams2.sum(axis=0)
fourgrams_freq2 = [(fourgram, sum_fourgrams[0, idx]) for fourgram, idx in fourgram_count_vect2.vocabulary_.items()]
fourgrams_freq2 =sorted(fourgrams_freq2, key = lambda x: x[1], reverse=True)

In [ ]:
number_of_fourgrams = 20
y_pos = np.arange(number_of_fourgrams)
objects = []
performance = []
for i in range(number_of_fourgrams):
    aux = fourgrams_freq2[i]
    objects.append(aux[0])
    performance.append(aux[1])
del number_of_fourgrams

In [ ]:
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Frequency')
plt.ylabel('4-grams')
plt.title('4-+ grams Frequency without stop words')
plt.show()

## Nuvens de Palavras

### Geral

In [ ]:
cloud = wordcloud.WordCloud(stopwords=Mystopwords+['de','da'], max_font_size=50, max_words=100, background_color="white").\
generate(' '.join(df['tweet_limpo'].to_list()).lower())
# Display the generated image:
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Anotados como crítica

In [ ]:
cloud = wordcloud.WordCloud(stopwords=Mystopwords, max_font_size=50, max_words=100, background_color="white").\
generate(' '.join(df[df['sent_manual']=='C']['tweet_limpo'].to_list()).lower())
# Display the generated image:
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Hierarchical clustering

In [ ]:
vec_words = TfidfVectorizer(stop_words=Mystopwords+\
                      ['of','rio','and','are','my','co','own','https','the','t','ex','at','rt','nunca','tudo',\
                      'alvaro','si'],
                             #max_df=1.0,
                             min_df=80,
                             #max_features=4000,
                      
                     )

In [ ]:
words_matrix = vec_words.fit_transform(df['tweet_limpo'])
words_matrix.shape

In [ ]:
words_matrix = words_matrix.transpose()
words_matrix.shape

In [ ]:
#plt.figure(figsize=(18, 16))
#plt.title("Word Dendogram")
#dend = shc.dendrogram(shc.linkage(words_matrix.todense(), method='ward'),
#                      labels=vec_words.get_feature_names())

fig = plt.figure()
plt.title("Word Dendogram")
ax = fig.add_subplot(1, 1, 1)
dend = shc.dendrogram(shc.linkage(words_matrix.todense(), method='ward'),
                      labels=vec_words.get_feature_names(),ax=ax)

ax.tick_params(axis='x', which='major', labelsize=20)
ax.tick_params(axis='y', which='major', labelsize=15)

### scatter text

In [ ]:
#https://github.com/JasonKessler/scattertext  #diversos exemplos

nlp = spacy.load('pt_core_news_sm')
corpus = st.CorpusFromPandas(df[df['sent_manual'].isin(['C','N'])],
                             category_col='sent_manual',
                             text_col='tweet_limpo',
                             nlp=nlp).build().remove_terms(Mystopwords, ignore_absences=True)

In [ ]:
html = st.produce_scattertext_explorer(corpus,
                                       category='C',
                                       category_name='Críticas',
                                       not_category_name='Não Críticas',
                                       width_in_pixels=1000,
                                       )
open("Convention-Visualization-geral.html", 'wb').write(html.encode('utf-8'))

In [ ]:
HTML(filename='Convention-Visualization-geral.html')

### Graph of words 

In [ ]:


#i = df.index[0]
#text =  np.array2string( df.loc[[i],['tweet_limpo']].values)
text = " ".join(df['tweet_limpo'].to_list())
bigrams = [(a, b) for l in [text] for a,b in zip(l.split(" ")[:-1], l.split(" ")[1:])]

# Create counter of words in clean bigrams
bigram_counts = Counter(bigrams)

# Create a network plot of grouped terms
bigram_df = pd.DataFrame(bigram_counts.most_common(50),
                             columns=['bigram', 'count'])

# Create dictionary of bigrams and their counts
d = bigram_df.set_index('bigram').T.to_dict('records')

# Create network plot 
G = nx.DiGraph()

# Create connections between nodes
for k, v in d[0].items():
    G.add_edge(k[0], k[1], weight=(v * 10))

fig, ax = plt.subplots(figsize=(12, 8))

pos = nx.spring_layout(G, k=5.5)
#pos = nx.kamada_kawai_layout(G)
#pos = nx.circular_layout(G)
# Plot networks
nx.draw_networkx(G, pos,
                 font_size=16,
                 width=3,
                 edge_color='gray',
                 node_color='purple',
                 with_labels = False,
                 ax=ax)

# Create offset labels
for key, value in pos.items():
    x, y = value[0]+.135, value[1]+.05
    ax.text(x, y,
            s=key,
            bbox=dict(facecolor='red', alpha=0.25),
            horizontalalignment='center', fontsize=14)
    
plt.show()

### Emojis

In [ ]:
#https://stackoverflow.com/questions/43146528/how-to-extract-all-the-emojis-from-text

import emoji
import regex

def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

In [ ]:
lista_emoji = []
for txt in df['tweet']:
    lista = split_count(txt)
    if len(lista)>0:
        lista_emoji.append(lista)
lista_emoji = [item for sublist in lista_emoji for item in sublist]
lista_emoji

emoji_counts = Counter(lista_emoji)

# Create a network plot of grouped terms
pd.DataFrame(emoji_counts.most_common(100), columns=['emoji', 'count'])
#matplotlib não plota emojis

## 2.2 ANÁLISE DE HASHTAGS 

### hashtags

In [ ]:
df.tweet.str.extractall(r'(\#\w+)')[0].value_counts().head(30).plot.bar()
print("Hashtags únicas:",df.tweet.str.extractall(r'(\#\w+)')[0].value_counts().shape[0])
print("Total de Hashtags:",df.tweet.str.extractall(r'(\#\w+)')[0].value_counts().sum())

### Associação entre hashtags 

* __Support__: This says how popular an itemset is, as measured by the proportion of transactions in which an itemset appears.
* __Confidence__: This says how likely item Y is purchased when item X is purchased, expressed as {X -> Y}. This is measured by the proportion of transactions with item X, in which item Y also appears.
* __Lift__: This says how likely item Y is purchased when item X is purchased, while controlling for how popular item Y is. A lift value greater than 1 means that item Y is likely to be bought if item X is bought, while a value less than 1 means that item Y is unlikely to be bought if item X is bought.

Fonte: https://www.kdnuggets.com/2016/04/association-rules-apriori-algorithm-tutorial.html

In [ ]:
lista_de_lista_dehashtags=[]
for tweet in df['tweet']:
    l = re.findall(r'(\#\w+)',tweet)
    if len(l)>1:
        lista_de_lista_dehashtags.append(l)

#aplica o algoritmo apriori
itemsets, rules = apriori(lista_de_lista_dehashtags, min_support=0.05,  min_confidence=0.5)
rules_rhs = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
    print(rule) 

## 2.3 ANÁLISE DE SENTIMENTOS 

### Métodos de dicionários 

In [ ]:
def polaridade(valor):
    if valor>0.02:
        return("positivo")
    elif valor<-0.02:
        return("negativo")
    else:
        return("neutro")

#### 'oplexicon3'

In [ ]:
col = 'sent_oplexicon3_ABP'
df[col].apply(polaridade).value_counts().plot.bar(title = col)

In [ ]:
df_aux = df[['data2', col]].copy()
df_aux['polaridade'] = df_aux[col].apply(polaridade)

ax = plt.gca()

df_aux[['data2', 'polaridade']][df_aux['polaridade']=='positivo']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = 'Positivo', ax=ax, color = 'blue')

df_aux[['data2', 'polaridade']][df_aux['polaridade']=='neutro']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = 'Neutro', ax=ax, color = 'black')

df_aux[['data2', 'polaridade']][df_aux['polaridade']=='negativo']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = 'Negativo', ax=ax, color = 'red')

ax.legend(['Positivo', 'Neutro', 'Negativo'])

plt.show()

#### 'SentiLex'

In [ ]:
col = 'sent_SentiLex_ABP'
df[col].apply(polaridade).value_counts().plot.bar(title = col)

In [ ]:
df_aux = df[['data2', col]].copy()
df_aux['polaridade'] = df_aux[col].apply(polaridade)

ax = plt.gca()

df_aux[['data2', 'polaridade']][df_aux['polaridade']=='positivo']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = 'Positivo', ax=ax, color = 'blue')

df_aux[['data2', 'polaridade']][df_aux['polaridade']=='neutro']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = 'Neutro', ax=ax, color = 'black')

df_aux[['data2', 'polaridade']][df_aux['polaridade']=='negativo']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = 'Negativo', ax=ax, color = 'red')

ax.legend(['Positivo', 'Neutro', 'Negativo'])

plt.show()

#### 'textblob_polarity'

In [ ]:
col = 'textblob_polarity'
df[col].apply(polaridade).value_counts().plot.bar(title = col)

In [ ]:
df_aux = df[['data2', col]].copy()
df_aux['polaridade'] = df_aux[col].apply(polaridade)

ax = plt.gca()

df_aux[['data2', 'polaridade']][df_aux['polaridade']=='positivo']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = 'Positivo', ax=ax, color = 'blue')

df_aux[['data2', 'polaridade']][df_aux['polaridade']=='neutro']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = 'Neutro', ax=ax, color = 'black')

df_aux[['data2', 'polaridade']][df_aux['polaridade']=='negativo']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = 'Negativo', ax=ax, color = 'red')

ax.legend(['Positivo', 'Neutro', 'Negativo'])

plt.show()

In [ ]:
df[df.select_dtypes(include=['float64']).columns].describe()

In [ ]:
#plt.figure(figsize=(10,6))
#sns.heatmap(df[df.select_dtypes(include=['float64']).columns].corr(), annot=True)
#plt.title('data correlations')

### Anotação Manual

In [ ]:
df['sent_manual'].value_counts().sort_values(ascending=False)

In [ ]:
df['sent_manual'].value_counts().sort_values(ascending=False).plot.bar()

In [ ]:
ax = plt.gca()

df[['data2', 'sent_manual']][df['sent_manual']=='C']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(linewidth=5, y = 'C', ax=ax, color = 'red')

df[['data2', 'sent_manual']][df['sent_manual']=='N']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(linewidth=5, y = 'N', ax=ax, color = 'black')

df[['data2', 'sent_manual']][df['sent_manual']=='E']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot( linewidth=5, y = 'N', ax=ax, color = 'blue')

ax.legend(['C', 'N', 'E'])

plt.show()


### críticas por usuário (anotadas manualmente) 

In [ ]:
df[df['sent_manual']=='C']['usuario'].value_counts().plot.bar(title='críticas por usuários')

### Machine Learning

#### MLP - Binary

In [ ]:
binary_vocab = pickle.load(open("MLP-binary-vocab", 'rb'))

In [ ]:
#DTM-BINÁRIA
# cria um objeto contador binário
binary_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}',binary=True, vocabulary=binary_vocab)
binary_vect.fit(df['tweet_limpo']) # treina o objeto nos textos processados
#print(binary_vect.get_feature_names())
#print(len(binary_vect.get_feature_names())," tokens")
#Transforma os documentos na matriz documento termo binária.
xvalid_binary =  binary_vect.transform(df['tweet_limpo'])

In [ ]:
import joblib
classifier = joblib.load(open("MLP-binary", 'rb'))


In [ ]:
predictions = classifier.predict(xvalid_binary)
Counter(predictions)

In [ ]:
df['MLP_binary'] = predictions
df['MLP_binary'] = df['MLP_binary'].apply(lambda x: "N" if x==1 else "C")

In [ ]:
df['MLP_binary'].value_counts().sort_values(ascending=False).plot.bar()

In [ ]:
ax = plt.gca()

df[['data2', 'MLP_binary']][df['MLP_binary']=='C']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(linewidth=5, y = 'C', ax=ax, color = 'red')

df[['data2', 'MLP_binary']][df['MLP_binary']=='N']['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(linewidth=5, y = 'N', ax=ax, color = 'black')

ax.legend(['C', 'N'])

plt.show()


In [ ]:
import datetime
df_aux = pd.DataFrame({
    "N":df[['data2', 'MLP_binary']][(df['MLP_binary']=='N') & (df['data2']>datetime.date(2020,3,3))]['data2'].value_counts().sort_index().asfreq('D').fillna(0).values,
    "C":df[['data2', 'MLP_binary']][df['MLP_binary']=='C']['data2'].value_counts().sort_index().asfreq('D').fillna(0).values},
    index = df[['data2', 'MLP_binary']][df['MLP_binary']=='C']['data2'].value_counts().sort_index().asfreq('D').fillna(0).index
)
df_aux['razao'] = df_aux["C"]/(df_aux["N"]+0.01)*100
df_aux.apply(lambda x: np.log(x + 0.01)).razao.plot()

## 2.4 MODELAGEM DE TÓPICOS

### Latent Dirichlet Analysis (LDA)



In [ ]:
corpus = df['tweet_limpo'].tolist()

In [ ]:
for i in range(0,len(corpus)):
    corpus[i]=corpus[i].lower()
    corpus[i] = re.sub('[0-9]+', '', corpus[i]) #remove numbers
    corpus[i] = re.sub(r'[^\w\s]','',corpus[i]) #remove punctuation
    corpus[i] = re.sub('\n','',corpus[i]) #remove \n - newline
    #corrige expressões comuuns
    corpus[i] = re.sub('paulo guedes','paulo_guedes',corpus[i]) 
    corpus[i] = re.sub('política monetária','política_monetária',corpus[i]) 
    corpus[i] = re.sub('roberto campos neto','roberto_campos_neto',corpus[i]) 
    corpus[i] = re.sub('reservas internacionais','reservas_internacionais',corpus[i]) 
    corpus[i] = re.sub('produto interno bruto','pib',corpus[i]) 
    corpus[i] = re.sub('instituição financeira','if',corpus[i]) 
    corpus[i] = re.sub('instituições financeiras','if',corpus[i]) 
    corpus[i] = re.sub('operação compromissada','operação_compromissada',corpus[i]) 
    corpus[i] = re.sub('operações compromissadas','operação_compromissada',corpus[i]) 
    corpus[i] = re.sub('relatório trimestral de inflação','rti',corpus[i]) 
    corpus[i] = re.sub('orçamento de guerra','orçamento_de_guerra',corpus[i]) 
    corpus[i] = re.sub('bi ','bilhões ',corpus[i]) 
    corpus[i] = re.sub('tri ','trilhões ',corpus[i]) 
    corpus[i] = re.sub('reunião do copom','copom',corpus[i]) 
    corpus[i] = re.sub('cheque especial','cheque_especial',corpus[i]) 
    corpus[i] = re.sub('conselho monetário nacional','cmn',corpus[i]) 
    corpus[i] = re.sub('relatório focus','focus',corpus[i]) 
    corpus[i] = re.sub('reclamações','reclamação',corpus[i]) 
    


In [ ]:
#carrega modelo pré-treinado para processar textos em português. Desabilita duas funções que não vamos usar
#nlp = spacy.load('pt_core_news_sm', disable=['parser', 'ner'])

#for i in range(0,len(corpus)): # varre a lista de textos
#    doc = nlp(corpus[i]) # executa um processamento de texto
#    corpus[i]=" ".join([token.lemma_ for token in doc]) # substitui o texto anterior por um texto contendo os lemas extraídos



In [ ]:
Mystopwords = Mystopwords + ['gt','bps','dias','tempo','cenário','evitar','presidente','acesse','site','texto','telefone','precisa','registre','ouvindo']

for i in range(0,len(corpus)): # varre a lista de textos
    words=corpus[i].split(" ") # separa o texto em palavras
    words_new = [w for w in words if w not in Mystopwords] #remove as stop words
    corpus[i] = ' '.join(words_new) # concantena as palavras novamente

corpus_NMF = corpus.copy()

In [ ]:
for idx in range(len(corpus)):
    corpus[idx] = [word for word in corpus[idx].split(' ') if word not in ['']];

#corpus[:2]

In [ ]:
Nr_topics = 5
id2word = gensim.corpora.Dictionary(corpus);
corpus_idx = [id2word.doc2bow(text) for text in corpus];

In [ ]:
lda = LdaModel(corpus=corpus_idx, id2word=id2word, num_topics=Nr_topics,passes=4,iterations=400)
#WARNING:gensim.models.ldamodel:too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy

In [ ]:
def get_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 10);
        word_dict['Topic # ' + '{:02d}'.format(i)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [ ]:
get_topics(lda, Nr_topics)

In [ ]:
doc_topics=[]
for doc in corpus_idx:
    lista_tuplas = lda.get_document_topics(doc)
    top_aux = 0
    prob_aux = 0
    for top, prob in lista_tuplas:
        if prob>prob_aux:
            top_aux = top
    doc_topics.append(top_aux)
df['lda_topico'] = doc_topics

In [ ]:
df['lda_topico'].value_counts().plot.bar()

In [ ]:
for i in range(Nr_topics):
    print("Tópico:", i)
    print(df[['usuario','lda_topico']][df['lda_topico']==i]['usuario'].value_counts().head(),"\n")

## Non-negative Matrix Factorization (NMF)

In [ ]:
count_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                             ngram_range=(1, 3),
                             #stop_words = Mystopwords, #NÃO TIRAR STOP WORDS, POIS JÁ FORAM RETIRADOS
                             #max_df = 10000, 
                             #min_df=10,
                             #max_features=10000
                            )
count_vect.fit(corpus_NMF) # treina o objeto nos textos processados
count_vect_dtm = count_vect.transform(corpus_NMF)
count_vect_dtm.shape
vocab = np.array(count_vect.get_feature_names())

In [ ]:
Nr_topics = 5

clf = decomposition.NMF(n_components=Nr_topics, random_state=1)

W1 = clf.fit_transform(count_vect_dtm)
H1 = clf.components_

In [ ]:
num_top_words=5

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' - '.join(t) for t in topic_words]

In [ ]:
show_topics(H1)

In [ ]:
def get_nmf_topics(model, feat_names, num_topics):
    '''
    Model = modelo treinado na função fit do NMF
    feat_names = lista de palavras obtidas de vectorizer.get_feature_names() (vocabulário)
    Nr_topics = número de tópicos
    '''
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [ ]:
get_nmf_topics(clf, vocab, Nr_topics)

In [ ]:
#W1.shape
df['nmf_topico'] = W1.argmax(axis=1)+1

In [ ]:
df['nmf_topico'].value_counts().plot.bar()

In [ ]:
for i in range(Nr_topics):
    print("Tópico:",i+1)
    print(show_topics(H1)[i])
    print(df[['usuario','nmf_topico']][df['nmf_topico']==i+1]['usuario'].value_counts().head(),"\n")

In [ ]:
ax = plt.gca()

df[['data2', 'nmf_topico']][df['nmf_topico']==1]['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = '1', ax=ax, color = 'red')

df[['data2', 'nmf_topico']][df['nmf_topico']==2]['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = '2', ax=ax, color = 'blue')

df[['data2', 'nmf_topico']][df['nmf_topico']==3]['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = '3', ax=ax, color = 'green')

df[['data2', 'nmf_topico']][df['nmf_topico']==4]['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = '4', ax=ax, color = 'magenta')

df[['data2', 'nmf_topico']][df['nmf_topico']==5]['data2'].value_counts().sort_index().asfreq('D').fillna(0).\
plot(xlim=('2020-03-01',np.max(df['data'])), linewidth=5, y = '5', ax=ax, color = 'black')

ax.legend(['1', '2', '3', '4','5'])

plt.show()


### Hierarchical LDA

In [ ]:
import tomotopy as tp


In [ ]:
mdl = tp.HLDAModel.load('models/hLDA.bin')

In [ ]:
for k in range(mdl.k):
    if not mdl.is_live_topic(k): continue
    print('Topic #{}'.format(k))
    print('Level', mdl.level(k))
    #print('Alive',mdl.is_live_topic(k))
    print('Nr Docs:', mdl.num_docs_of_topic(k))
    print('Parent Topics:')
    print(mdl.parent_topic(k))
    print('Children Topics:')
    print(mdl.children_topics(k))
    for word, prob in mdl.get_topic_words(k):
        print('\t', word, prob, sep='\t')


In [ ]:
df_aux = pd.read_pickle('dados\\df_hlda_topics.pkl')
df = df.merge(df_aux, how='left', left_index=True, right_index=True)


In [ ]:
df['hlda_topics'].value_counts().head(20).plot.bar()

In [ ]:
#REDE DO HLDA COM NÓS SENDO OS TÓPICOS E OS VÉRTICES SENDO AS RELAÇÕES DE HIERARQUIA ENTRE OS TÓPICOS
Ghlda = nx.Graph()   
for k in range(mdl.k):
    if not mdl.is_live_topic(k): continue
    lista_top = mdl.children_topics(0).tolist()
    if len(lista_top)>0:
        for l in lista_top:
            Ghlda.add_edge(k, l)           

In [ ]:
nx.draw_kamada_kawai(Ghlda, with_labels=True,font_size=30)

## 2.5 CLUSTERING

Baseado em https://github.com/twitterdev/do_more_with_twitter_data/blob/master/examples/clustering_users/clustering-users.ipynb

In [ ]:
vec_tweets = TfidfVectorizer(stop_words=Mystopwords+\
                      ['of','rio','and','are','my','co','own','https','the','t','ex','at','rt','nunca','tudo',\
                      'alvaro','si'],
                             #max_df=1.0,
                             min_df=20,
                             #max_features=4000,
                      
                     )

In [ ]:
#tweet_matrix = vec_tweets.fit_transform(unique_tweets)
tweet_matrix = vec_tweets.fit_transform(df['tweet_limpo'])

In [ ]:
tweet_matrix.shape

In [ ]:
df.shape

In [ ]:
hdbs_tweets = hdbscan.HDBSCAN(min_cluster_size=25,
                               prediction_data=True,
                               core_dist_n_jobs=-1,
                               memory='data')
hdbs_tweets.fit(tweet_matrix.todense())

In [ ]:
# get the population sizes 
label_counts = Counter(hdbs_tweets.labels_)
xs, ys = [], []
for k,v in label_counts.items():
    xs.append(k)
    ys.append(v)

# draw the chart
plt.bar(xs, ys)

plt.xticks(range(-1, len(label_counts)))
plt.ylabel('population')
plt.xlabel('cluster label')
plt.title('population sizes ({} clusters found by hdbscan)'.format(len(label_counts) - 1));

In [ ]:
print("Número de clusters:", hdbs_tweets.labels_.max()+1 )
print("(cluster, qtd)")
([(xs[i], ys[i]) for i in range(len(xs))])

In [ ]:
strongest_features(hdbs_tweets, vec_tweets, tweet_matrix, topk=10)

In [ ]:
df['cluster'] = hdbs_tweets.labels_

In [ ]:
def strongest_features_per_cluster(cluster, model, vectorizer, matrix, topk=10):
    """
    Helper function to display a simple text representation of the top-k most 
    important features in our fit model and vectorizer.
    
    cluster: cluster number
    matrix: matriz obtida da função vec.fit_transform
    vectorizer: sklearn vectorizer
    topk: k numbers of words to get per cluster
    
    """
    features = vectorizer.get_feature_names()
    # ignore noise labels
    relevant_labels = [ x for x in set(model.labels_) if x >= 0 ]
    this_label = cluster
    matching_rows = np.where(hdbs.labels_ == this_label)[0]
    coeff_sums = np.sum(matrix[matching_rows], axis=0).A1
    sorted_coeff_idxs = np.argsort(coeff_sums)[::-1]
    print('Cluster {}: '.format(this_label), end='')
    for idx in sorted_coeff_idxs[:topk]:
        print('{} '.format(features[idx]), end='')
    print()
 

In [ ]:
for i in range(-1,hdbs_tweets.labels_.max()+1):
    #print("Cluster:",i)
    print(strongest_features_per_cluster(i,hdbs_tweets,vec_tweets,tweet_matrix,10))
    print(df[['usuario','cluster']][df['cluster']==i]['usuario'].value_counts().head(),"\n")

## 2.6 NER 
Named Entity Recognition

In [ ]:
nlp = spacy.load('pt_core_news_sm', disable=['tagger', 'parser', 'textcat'])

In [ ]:
def get_NER(text):
    doc = nlp(text)
    return [(e.text, e.label_) for e in doc.ents]

In [ ]:
df['entities'] = df['tweet'].apply(get_NER)

In [ ]:
df[['tweet','entities']].head(3)

In [ ]:
lista_entities = df['entities'].apply(pd.Series).stack().reset_index(drop=True)
len(lista_entities)

In [ ]:
Counter(lista_entities).most_common(20)

In [ ]:
lista_entities2 = [ent for ent,tipo in lista_entities]

In [ ]:
Counter(lista_entities2).most_common(20)

# 3. ANÁLISE DE REDES

## Análise de redes - Grafos  

https://github.com/ericmjl/Network-Analysis-Made-Simple/

https://github.com/ericmjl/nxviz/tree/master/examples

In [ ]:
def path_exists(node1, node2, G):
    """
    This function checks whether a path exists between two nodes (node1, 
    node2) in graph G.
    
    Special thanks to @ghirlekar for suggesting that we keep track of the 
    "visited nodes" to prevent infinite loops from happening. This also 
    removes the need to remove nodes from queue.
    
    Reference: https://github.com/ericmjl/Network-Analysis-Made-Simple/issues/3
    
    With thanks to @joshporter1 for the second bug fix. Originally there was 
    an extraneous "if" statement that guaranteed that the "False" case would 
    never be returned - because queue never changes in shape. Discovered at 
    PyCon 2017.
    
    With thanks to @chendaniely for pointing out the extraneous "break".
    
    If you would like to see @dgerlanc's implementation, see 
    https://github.com/ericmjl/Network-Analysis-Made-Simple/issues/76
    """
    visited_nodes = set()
    queue = [node1]
    
    for node in queue:
        neighbors = list(G.neighbors(node))
        if node2 in neighbors:
            print('Path exists between nodes {0} and {1}'.format(node1, node2))
            return True
        else:
            visited_nodes.add(node)
            queue.extend([n for n in neighbors if n not in visited_nodes])
    
    print('Path does not exist between nodes {0} and {1}'.format(node1, node2))
    return False

def extract_path_edges(G, source, target):
    # Check to make sure that a path does exists between source and target.
    if nx.has_path(G, source, target):
        nodes = nx.shortest_path(G, source, target)
        newG = G.subgraph(nodes)
        return newG

    else:
        raise Exception('Path does not exist between nodes {0} and {1}.'.format(source, target))

def extract_neighbor_edges(G, node):
    neighbors = list(G.neighbors(node))
    newG = nx.Graph()
    
    for n1, n2 in G.edges():
        if (n1 == node and n2 in neighbors) or (n1 in neighbors and n2 == node):
            newG.add_edge(n1, n2)
            
    return newG


def extract_neighbor_edges2(G, node):
    neighbors = G.neighbors(node)
    newG = nx.Graph()
    
    for neighbor in neighbors:
        if (node, neighbor) in G.edges() or (neighbor, node) in G.edges():
            newG.add_edge(node, neighbor)

    return newG

def get_triangles(G, node):
    neighbors1 = set(G.neighbors(node))
    triangle_nodes = set()
    triangle_nodes.add(node)
    """
    Fill in the rest of the code below.
    """
    for nbr1, nbr2 in itertools.combinations(neighbors1, 2):
        if G.has_edge(nbr1, nbr2):
            triangle_nodes.add(nbr1)
            triangle_nodes.add(nbr2)
    return triangle_nodes

In [ ]:
df_g= pd.read_pickle('dados\\df_tweet_grafo.pkl')
df_g.info()

In [ ]:
df_g = df_g[df_g['usuario1'].isin(lista_usuarios2)][df_g['usuario2'].isin(lista_usuarios2)]\
[df_g['data']>"2020-02-29"]
df_g.info()

In [ ]:
df_g['tipo'].value_counts()

In [ ]:
GT = nx.from_pandas_edgelist(df_g, 'usuario1', 'usuario2', ['likes', 'retweets','tipo'], create_using=nx.DiGraph())
nx.write_gml(GT,"dados\\grafo_geral.gml")
dic = {}
for user in df['usuario'].unique():
    dic[user] = df[df['usuario']==user]['tipo_usuario'].values[0]
nx.set_node_attributes(GT, values=dic, name='tipo_usuario')
print(nx.info(GT))

## 3.1 ANÁLISE DE TOPOLOGIA 

In [ ]:
nx.is_directed(GT)

In [ ]:
nx.is_strongly_connected(GT)

In [ ]:
nx.is_weakly_connected(GT)

In [ ]:
nx.is_weighted(GT)

In [ ]:
#nx.is_aperiodic(GT)
#muitos warnings
#False

In [ ]:
nx.classes.density(GT)
#The density is 0 for a graph without edges and 1 for a complete graph.

In [ ]:
nx.reciprocity(GT)

In [ ]:
nx.algorithms.cluster.transitivity(GT)

In [ ]:
nx.degree_assortativity_coefficient(GT)

In [ ]:
nx.degree_pearson_correlation_coefficient(GT)

In [ ]:
#list(nx.algorithms.community.greedy_modularity_communities(GT))
#IndexError: list index out of range


In [ ]:
print("isolates: ", nx.algorithms.isolate.number_of_isolates(GT))
if nx.algorithms.isolate.number_of_isolates(GT)>0:
    list(nx.isolates(GT))

In [ ]:
nx.networkx.algorithms.connectivity.connectivity.node_connectivity(GT)

In [ ]:
nx.draw(GT, with_labels=True,font_size=30)

## 3.2 ANÁLISE DE CENTRALIDADE 

### Medidas de centralidade

#### GRAU

In [ ]:
# Mariores Graus.
sorted(GT.degree(), key=lambda x:x[1], reverse=True)[:20]

In [ ]:
# Degree Centrality
sorted(nx.degree_centrality(GT).items(), key=lambda x: x[1], reverse=True)[:20]

In [ ]:
# Degree Centrality
sorted(nx.in_degree_centrality(GT).items(), key=lambda x: x[1], reverse=True)[:20]

In [ ]:
# Degree Centrality
sorted(nx.out_degree_centrality(GT).items(), key=lambda x: x[1], reverse=True)[:20]

#### CLOSENESS

In [ ]:
# closeness
closeness_centrality = nx.closeness_centrality(GT) 
sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:20]

#### BETWEENESS

In [ ]:
# betweeness
betweeness_centrality = nx.betweenness_centrality(GT)
sorted(betweeness_centrality.items(), key=lambda x: x[1], reverse=True)[:20]

#### CENTRALIDADE DE AUTOVETOR

In [ ]:
# eigenvector_centrality
eigenvector_centrality = nx.eigenvector_centrality_numpy(GT)

# normaliza o eigenvector_centrality
max_value = max(eigenvector_centrality.items(), key=lambda x: x[1])

ec_scaled = {}
for k in eigenvector_centrality.keys():
    ec_scaled[k] = eigenvector_centrality[k] / max_value[1]

# Scaled by the most central character (karev)
sorted(ec_scaled.items(), key=lambda x:x[1], reverse=True)[:20]

#### PAGERANK

In [ ]:
#nx.pagerank(GT, alpha=0.9)
sorted(nx.pagerank(GT, alpha=0.9).items(), key=lambda x:x[1], reverse=True)[:20]

#### KATZ

In [ ]:
# katz_centrality
katz_centrality = nx.katz_centrality_numpy(GT)

# normaliza o eigenvector_centrality
max_value = max(katz_centrality.items(), key=lambda x: x[1])

ec_scaled = {}
for k in katz_centrality.keys():
    ec_scaled[k] = katz_centrality[k] / max_value[1]

# Scaled by the most central character (karev)
sorted(ec_scaled.items(), key=lambda x:x[1], reverse=True)[:20]

## 3.3 ANÁLISE DE COMUNIDADES

In [ ]:
#DEMORA
gn_comm = girvan_newman(GT)
first_iteration_comm = tuple(sorted(c) for c in next(gn_comm))

In [ ]:

def create_color_map(G, attribute, seaborn_palette="colorblind"):
    """Return a list of hex color mappings for node attributes"""
    attributes = [G.nodes[label][attribute] for label in G.nodes()]

    # get the set of possible attributes
    attributes_unique = list(set(attributes))
    num_values = len(attributes_unique)

    # generate color palette from seaborn
    palette = sns.color_palette(seaborn_palette, num_values).as_hex()

    # create a mapping of attribute to color
    color_map = dict(zip(attributes_unique, palette))

    # map the attribute for each node to the color it represents
    node_colors = [color_map[attribute] for attribute in attributes]

    return node_colors, color_map, palette

def map_communities(G, communities):
    """Return a mapping of community membership from a community set tuple"""

    community_map = {}
    for node in G.nodes():
        for i, comm in enumerate(communities):
            if node in comm:
                community_map[node] = i
        if community_map.get(node, None) is None:
            community_map[node] = None
    return community_map

In [ ]:
community_map = map_communities(GT, first_iteration_comm)
nx.set_node_attributes(GT, name='community', values=community_map)
node_colors, color_map, palette = create_color_map(GT, 'community')
print("Número de comunidades: ", max(community_map.values())+1)

In [ ]:
nx.draw(GT, node_color=node_colors, with_labels=True,font_size=30)

In [ ]:
nx.draw_kamada_kawai(GT,node_color=node_colors, with_labels=True,font_size=30)

In [ ]:
sorted(community_map.items(), key=lambda x:x[1], reverse=True)

In [ ]:
def retorna_community_girvan_newman(user):
    if user in community_map.keys():
        return community_map[user] + 1
    else:
        return np.nan
    
df['girvan_newman'] = df['usuario'].apply(retorna_community_girvan_newman)

In [ ]:
nodes = (
    node
    for node, data
    in GT.nodes(data=True)
    if data.get("community") == 1
)
nx.draw(GT.subgraph(nodes),with_labels=True,font_size=30)

### Detecção de comunidades 2

Clauset-Newman-Moore greedy modularity maximization

É necessário converter o gráfico para indireto.

In [ ]:
c = list(greedy_modularity_communities(GT.to_undirected()))
print("Nr de comunidades:", len(c),"\n")
dict_modularity = {}
for i, l in enumerate(c, start = 1):
    print("comunidade ",i," ",l,"\n")
    for u in l:
        dict_modularity[u] = i

In [ ]:
def retorna_community_modularity(user):
    if user in dict_modularity.keys():
        return dict_modularity[user]
    else:
        return np.nan

df['modularity'] = df['usuario'].apply(retorna_community_modularity)

In [ ]:
print("5 maiores comunidades")
c = Counter(list(dict_modularity.values())).most_common(5)
c[0:5]

In [ ]:
for i in range(3):
    j = c[i][0]
    print("Comunidade:", j,"\nTóp - Qtd")
    print(df[['modularity','nmf_topico']][df['modularity']==j]['nmf_topico'].value_counts().head(),"\n")

### Detecção de comunidades 3
(conversão para grafo indireto)

Fonte: https://www.analyticsvidhya.com/blog/2020/04/community-detection-graphs-networks/

In [ ]:
def edge_to_remove(graph):
    G_dict = nx.edge_betweenness_centrality(graph)
    edge = ()

    # extract the edge with highest edge betweenness centrality score
    for key, value in sorted(G_dict.items(), key=lambda item: item[1], reverse = True):
        edge = key
        break

    return edge

def girvan_newman(graph):
	# find number of connected components
	sg = nx.connected_components(graph)
	sg_count = nx.number_connected_components(graph)

	while(sg_count == 1):
		graph.remove_edge(edge_to_remove(graph)[0], edge_to_remove(graph)[1])
		sg = nx.connected_components(graph)
		sg_count = nx.number_connected_components(graph)

	return sg

In [ ]:
# find communities in the graph
c = girvan_newman(GT.to_undirected(reciprocal=True).copy())

# find the nodes forming the communities
node_groups = []

for i in c:
    node_groups.append(list(i))
    
node_groups

# 4. DETECÇÃO DE EVENTOS

https://github.com/davidzchen-ut/TwitterEvents/blob/master/TwitterEvents.ipynb

In [ ]:
df2 = df[df['data']>"2020-03-01"][['data','data2','tweet','tweet_limpo']].copy()
df2['Datetime']= pd.to_datetime(df2['data2'].apply(str)+' '+df2['data'].apply(lambda x: x.hour).apply(str)+':00')

tableFlag=[]

count = 0
# iterating through clusters and finding hourly count of tweets

dfchange = df2
ts = dfchange.set_index('Datetime')
vc = ts.groupby('Datetime').count()
col = ['tweet']
vc2 = vc[col]
##calculating threshold to flag hours with more tweets
th = vc2['tweet'].mean()+2*vc2['tweet'].std()
#iterating through each row to find flags
for index, row in vc2.iterrows():
    if (row["tweet"]>th):
        tableFlag.append([index])
        count+=1

print(count)
#tableFlag

#"""

In [ ]:
# graphing the threshold vs hourly tweet occurences
#dfchange = df2
#ts = dfchange.set_index('Datetime')
#vc = ts.groupby('Datetime').count()
#col = ['tweet']
#vc2 = vc[col]
vc3 = vc2.copy()
#th = vc2['tweet'].mean()+2*vc2['tweet'].std()
#for index, row in vc2.iterrows():
#    if (row["tweet"]>th):
#        tableFlag.append([index])
#        count+=1

vc4 = vc2.copy()
vc4.rename(columns={'tweet':'Hourly'},inplace=True)
vc3['tweet'] = th
vc3.rename(columns={'tweet':'Threshold'},inplace=True)

ax = vc2.plot()

#ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
vc3.plot(ax = ax)

In [ ]:
# retrieve tweets associated with flagged timestamp
df2['event_id'] = -1
for index, array in enumerate(tableFlag):
    #cluster = array[0]
    timestamp = array[0]
    print(array[0])
    df2['event_id'] = (np.where((df2['Datetime'] == timestamp), index, df2["event_id"]))

In [ ]:
events = df2

nr_spikes = np.max(events['event_id'])+1
# Find all tweets that created each spike (950 spikes in our case)
tweetsPerEvent = []
for i in range(0, nr_spikes):
    tweetsPerEvent.append(events[events['event_id'] == i])

### Find Similar Tweets

In [ ]:
num_clusters = 10
#stemmer = SnowballStemmer("portuguese")

In [ ]:
#Get TF-IDF and Similarity Matrices
def getTFIDFAndSim(tweets):
    vect = TfidfVectorizer(max_df=0.8, max_features=200000, stop_words=Mystopwords, use_idf=True, ngram_range=(1,3))
    tfidf = vect.fit_transform(tweets)
    sim = (tfidf * tfidf.T).A
    return tfidf, sim

#Find Clusters of Similar Tweets
def findSimilarTweets(tweets, tfidf):
    clustering = DBSCAN(eps=1.2, min_samples=2).fit(tfidf)
    #clustering = hdbscan.HDBSCAN(min_cluster_size=2).fit(tfidf)

    clusters = clustering.labels_.tolist()
    temp = clusters.copy()
    temp.remove

    addedCluster = tweets.copy()
    addedCluster['Cluster'] = clusters

    #pd.options.display.max_colwidth = 100

    cls = []
    for i in range(0, num_clusters):
        tweetsInCluster = addedCluster[addedCluster['Cluster'] == i]
        cls.append(tweetsInCluster)
        
        if(tweetsInCluster.shape[0] > 1): #há tweets repetidos, por isso > 2
            #print("Cluster", i, ":", tweetsInCluster.shape[0])
            print("Evento:", tweetsInCluster['event_id'].to_list()[0])
            print("Data-hora:", tweetsInCluster['Datetime'].to_list()[0])
            for txt in tweetsInCluster['tweet'].to_list()[:10]:
                print(txt)
            return True
    
    return False
    #return cls

In [ ]:
count = 0
#df = pd.DataFrame();
    
# For each spike, clean the tweets, stem and tokenize the tweets, and find the TF-IDF vectors for the tweets. Then cluster
# on the vectors to find similar tweets. If clusters are found, then take the largest cluster to find the tweets that define
# the event.
for i in range(0, len(tweetsPerEvent)):
    if(tweetsPerEvent[i].shape[0] < th):
        continue
    
    count += 1
    #tweets = cleanData(tweetsPerEvent[i])
    #tweetsContent = tweets.copy()['tweet']
    tweets = tweetsPerEvent[i]
    tweetsContent = tweets.copy()['tweet_limpo']
    tfidf, sim = getTFIDFAndSim(tweetsContent)
    #totalvocab_stemmed, totalvocab_tokenized = stemAndTokenize(tweetsContent)
    #print("Event", i, ":")
    #print(tweetsPerEvent[i][['data','tweet_limpo']])
    #if(findSimilarTweets(tweets, tfidf)):
    #    df = pd.concat([df, tweetsPerEvent[i]])
    findSimilarTweets(tweets, tfidf)
    print("\n")
    


# 5. TENDÊNCIAS

In [ ]:
df_test = df[['data','tweet_limpo']].copy()
df_test.columns = ['data','tweet']
df_test['data'] = df_test['data'].apply(lambda x: x.date())

df_test2 = df_test.groupby(['data'])['tweet'].apply(lambda x: ' '.join(x)).reset_index().sort_values(by='data')
df_test2.set_index('data', inplace=True)
df_test2['tweet'] = df_test2['tweet'].apply(lambda x: x.lower())
df_test2['tweet'] = df_test2['tweet'].apply(lambda x: re.sub('\n', ' ', x))

#df_test2 = df_test2.tweet.str.split(expand=True).stack().value_counts()

i = df_test2.index[0]
df_aux = df_test2.loc[[i]]['tweet'].str.split(expand=True).stack().value_counts().to_frame()
df_aux.columns = ['qtd']
df_aux['data'] = df_aux['qtd'].apply(lambda x: i)
df_aux = df_aux.reset_index()
df_aux.columns = ['palavra', 'qtd', 'data']

for i in df_test2.index[1:]:
    df_aux2 = df_test2.loc[[i]]['tweet'].str.split(expand=True).stack().value_counts().to_frame()
    df_aux2.columns = ['qtd']
    df_aux2['data'] = df_aux2['qtd'].apply(lambda x: i)
    df_aux2 = df_aux2.reset_index()
    df_aux2.columns = ['palavra', 'qtd', 'data']
    df_aux = pd.concat([df_aux, df_aux2])
    


#BIGRAMAS

i = df_test2.index[0]
text =  np.array2string( df_test2.loc[[i],['tweet']].values)
text = re.sub('\[',' ',text)
text = re.sub('\]',' ',text)
bigrams = [a+' '+b for l in [text] for a,b in zip(l.split(" ")[:-1], l.split(" ")[1:])]
bigrams = Counter(bigrams)
df_aux2 = pd.DataFrame.from_dict(bigrams, orient='index').reset_index()
df_aux2.columns = ['palavra','qtd']
df_aux2 = df_aux2[ df_aux2['palavra']  != ' ']
df_aux2['data'] = df_aux2['qtd'].apply(lambda x: i)
df_aux = pd.concat([df_aux, df_aux2])

for i in df_test2.index[1:]:
    text =  np.array2string( df_test2.loc[[i],['tweet']].values)
    text = re.sub('\[',' ',text)
    text = re.sub('\]',' ',text)
    bigrams = [a+' '+b for l in [text] for a,b in zip(l.split(" ")[:-1], l.split(" ")[1:])]
    bigrams = Counter(bigrams)
    df_aux2 = pd.DataFrame.from_dict(bigrams, orient='index').reset_index()
    df_aux2.columns = ['palavra','qtd']
    df_aux2 = df_aux2[ df_aux2['palavra']  != ' ']
    df_aux2['data'] = df_aux2['qtd'].apply(lambda x: i)
    df_aux = pd.concat([df_aux, df_aux2])

#TRIGRAMAS

i = df_test2.index[0]
text =  np.array2string( df_test2.loc[[i],['tweet']].values)
text = re.sub('\[',' ',text)
text = re.sub('\]',' ',text)
trigrams = [a+' '+b+' '+c for l in [text] for a,b,c in zip(l.split(" ")[:-2], l.split(" ")[1:-1], l.split(" ")[2:])]
trigrams = Counter(trigrams)
df_aux2 = pd.DataFrame.from_dict(trigrams, orient='index').reset_index()
df_aux2.columns = ['palavra','qtd']
df_aux2 = df_aux2[ df_aux2['palavra']  != ' ']
df_aux2['data'] = df_aux2['qtd'].apply(lambda x: i)
df_aux = pd.concat([df_aux, df_aux2])

for i in df_test2.index[1:]:
    text =  np.array2string( df_test2.loc[[i],['tweet']].values)
    text = re.sub('\[',' ',text)
    text = re.sub('\]',' ',text)
    trigrams = [a+' '+b+' '+c for l in [text] for a,b,c in zip(l.split(" ")[:-2], l.split(" ")[1:-1], l.split(" ")[2:])]
    trigrams = Counter(trigrams)
    df_aux2 = pd.DataFrame.from_dict(trigrams, orient='index').reset_index()
    df_aux2.columns = ['palavra','qtd']
    df_aux2 = df_aux2[ df_aux2['palavra']  != ' ']
    df_aux2['data'] = df_aux2['qtd'].apply(lambda x: i)
    df_aux = pd.concat([df_aux, df_aux2])

df_ngrams = df_aux.copy()

del df_test, df_test2, df_aux, df_aux2#, bigrams, trigrams

#transformação de qtd em indice = nr_expressoes/nr_tweets

df_aux = df['data'].apply(lambda x: x.date()).value_counts().to_frame().reset_index()
df_aux.columns = ['data','nr']
df_ngrams = df_ngrams.merge(df_aux , how='left', on='data')
df_ngrams['indice'] = df_ngrams['qtd']/df_ngrams['nr']
del df_aux

df_ngrams = df_ngrams[['palavra','data','indice']]


In [ ]:
df_ngrams[df_ngrams['palavra'] == 'copom'].set_index('data').sort_index().asfreq('D').fillna(0).\
plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'] == 'paulo guedes'].set_index('data').sort_index().asfreq('D').fillna(0).\
plot(linewidth = 10)# pg

In [ ]:
df_ngrams[df_ngrams['palavra'] == 'roberto campos'].set_index('data').sort_index().asfreq('D').fillna(0).\
plot(linewidth = 10) #rcn

In [ ]:
df_ngrams[df_ngrams['palavra'] == 'reservas'].set_index('data').sort_index().asfreq('D').fillna(0).\
plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(['bcb','banco central','bacen']))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(['cmn','conselho monetário nacional']))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(['política monetária','politica monetaria']))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(['coronavirus','covid-19','coronavírus','covid19','corona virus','corona vírus','pandemia','covid','ncov19','ncov2019']))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(['dólar','dólares','dolar','dolares','câmbio','política cambial']))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(['ipca','inflação','deflação']))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(['juros','selic','copom','di']))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(["incerteza","incertezas","incerto","incerta","dúvida","dúvidas","hesitação", 
                                                      "imprecisão", "indecisão", "indefinição", "indeterminação", "insegurança", 
                                                      "interrogação","desconfiança"]))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(["crise","crises","recessão"]))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(['focus']))].groupby('data').\
sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(['pib','produto interno bruto']))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(["desemprego","desempregado","desempregada","desempregados","desempregadas","emprego","empregos"]))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(['congresso nacional','senado','câmara','pec']))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

In [ ]:
df_ngrams[df_ngrams['palavra'].str.contains('|'.join(['regulação','norma','resolução','circular']))].\
groupby('data').sum().sort_index().asfreq('D').fillna(0).plot(linewidth = 10)

# 6. INFORMATION RETRIEVAL

## Querying text data with TF-IDF

In [ ]:

# Provide your own query terms here
def query_text(QUERY_TERMS,nr_retornos = 5):

    

    # Load in human language data from wherever you've saved it
    #DATA = 'resources/ch05-textfiles/ch05-timoreilly.json'
    #data = json.loads(open(DATA).read())

    #activities = [post['content'].lower().split() 
    #              for post in data 
    #                if post['content'] != ""]

    activities = [txt.lower().split() for txt in df['tweet_limpo'] ]
    activities_idx = [idx for idx in df['tweet_limpo'].index ]


    # TextCollection provides tf, idf, and tf_idf abstractions so
    # that we don't have to maintain/compute them ourselves

    tc = nltk.TextCollection(activities)

    relevant_activities = []

    for idx in range(len(activities)):
        score = 0
        for term in [t.lower() for t in QUERY_TERMS]:
            score += tc.tf_idf(term, activities[idx])
        if score > 0:
            relevant_activities.append({'score': score, 'idx' : activities_idx[idx], 'tweet': activities[idx]})

    # Sort by score and display results

    relevant_activities = sorted(relevant_activities,
                                 key=lambda p: p['score'], reverse=True)
    for activity in relevant_activities[:nr_retornos]:
        print('Index: {0}'.format(activity['idx']))
        print('Tweet: {0}'.format(' '.join(activity['tweet'])))
        print('Author: {0}'.format(df[df.index == activity['idx']]['usuario'].values[0]))
        print('Data: {0}'.format(df[df.index == activity['idx']]['data'].values[0]))
        print('Score: {0}'.format(activity['score']))
        print()

In [ ]:
query_text(QUERY_TERMS = ['covid','coronavirus','coronavírus'])

In [ ]:
query_text(QUERY_TERMS = ['copom'])

In [ ]:
query_text(['política','monetária'])

In [ ]:
query_text(['autonomia','banco', 'central'])

In [ ]:
query_text(['reclamar','reclamação','denuncia','denúncia'])

In [ ]:
query_text(['crise','crises','recessão'])

In [ ]:
query_text(['boletim','focus'])

# 7. Resultados de análises das críticas

## Críticas com mais likes 

In [ ]:
df[df.MLP_binary == 'C'][['usuario','tweet','likes']].sort_values(by = 'likes', ascending=False).head(5)['tweet'].values

## Críticas com mais retweets 

In [ ]:
df[df.MLP_binary == 'C'][['usuario','tweet','retweets']].sort_values(by = 'retweets', ascending=False).head(5)['tweet'].values

## Usuários mais críticos proporcionalmente

In [ ]:
df_aux = df.usuario.value_counts()
df_aux = df_aux.to_frame().merge(df[df.MLP_binary == 'C']['usuario'].value_counts(), how='left', left_index=True, right_index=True)
df_aux['razao'] = df_aux['usuario_y']/df_aux['usuario_x']

In [ ]:
df_aux[df_aux.usuario_y>10].sort_values(by = 'razao', ascending=False)

In [ ]:
usuario_mais_critico = df_aux[df_aux.usuario_y>10].sort_values(by = 'razao', ascending=False).index[0]
usuario_mais_critico

In [ ]:
print(df[(df.MLP_binary == 'C') & (df.usuario == usuario_mais_critico)]['tweet'].values[:10])

In [ ]:
usuario_mais_critico2 = df_aux[df_aux.usuario_y>10].sort_values(by = 'razao', ascending=False).index[1]
usuario_mais_critico2

In [ ]:
print(df[(df.MLP_binary == 'C') & (df.usuario == usuario_mais_critico2)]['tweet'].values[:10])

## Tópicos mais criticados proporcionalmente

In [ ]:
df_aux = df.hlda_topics.value_counts()
df_aux
df_aux = df_aux.to_frame().merge(df[df.MLP_binary == 'C']['hlda_topics'].value_counts(), how='left', left_index=True, right_index=True)
df_aux['razao'] = df_aux['hlda_topics_y']/df_aux['hlda_topics_x']

In [ ]:
df_aux[df_aux.hlda_topics_y>10].sort_values(by = 'razao', ascending=False)

In [ ]:
topico_mais_criticado = df_aux[df_aux.hlda_topics_y>10].sort_values(by = 'razao', ascending=False).index[0]
mdl.get_topic_words(int(topico_mais_criticado))

In [ ]:
print(df[(df.MLP_binary == 'C') & (df.hlda_topics == topico_mais_criticado)]['tweet'].values[:5])
#df[(df.MLP_binary == 'C') & (df.hlda_topics == topico_mais_criticado)]

In [ ]:
topico_mais_criticado2 = df_aux[df_aux.hlda_topics_y>10].sort_values(by = 'razao', ascending=False).index[1]
mdl.get_topic_words(int(topico_mais_criticado2))

In [ ]:
print(df[(df.MLP_binary == 'C') & (df.hlda_topics == topico_mais_criticado2)]['tweet'].values[:5])

## Comunidades mais críticas proporcionalmente

In [ ]:
df_aux = df.modularity.value_counts()
df_aux
df_aux = df_aux.to_frame().merge(df[df.MLP_binary == 'C']['modularity'].value_counts(), how='left', left_index=True, right_index=True)
df_aux['razao'] = df_aux['modularity_y']/df_aux['modularity_x']

In [ ]:
df_aux[df_aux.modularity_y>10].sort_values(by = 'razao', ascending=False)

In [ ]:
comunidade_mais_critica = df_aux[df_aux.modularity_y>10].sort_values(by = 'razao', ascending=False).index[0]
df[df.modularity == comunidade_mais_critica]['usuario'].unique()

In [ ]:
print(df[(df.MLP_binary == 'C') & (df.modularity == comunidade_mais_critica)]['tweet'].values[:5])
#df[(df.MLP_binary == 'C') & (df.modularity == comunidade_mais_critica)]

In [ ]:
comunidade_mais_critica2 = df_aux[df_aux.modularity_y>10].sort_values(by = 'razao', ascending=False).index[1]
df[df.modularity == comunidade_mais_critica2]['usuario'].unique()

In [ ]:
print(df[(df.MLP_binary == 'C') & (df.modularity == comunidade_mais_critica2)]['tweet'].values[:5])

In [ ]:
df.columns

Por assuntos específicos

In [ ]:
print("Críticas à cédula de 200 reais")
df_aux = df[df['MLP_binary']=='C']
df_aux[df_aux['tweet'].str.contains('|'.join(['cédula','200 ']))]['tweet'].values[1:3]

In [ ]:
print("Críticas ao PIX")
df_aux = df[df['MLP_binary']=='C']
df_aux[df_aux['tweet'].str.contains('|'.join(['pix','PIX','instantâneo']))]['tweet'].values

In [ ]:
print("Críticas à liberação do compulsório")
df_aux = df[df['MLP_binary']=='C']
df_aux[df_aux['tweet'].str.contains('|'.join(['liberação','compulsório']))]['tweet'].values

## Análise de impacto

Copom 

In [ ]:
filtro_copom = ['(S|s)elic','(C|c)opom','(J|j)uros','SELIC','JUROS','(C|c)omunicado','(A|a)ta']

In [ ]:
df[(df.data>"2020-03-18 16:00:00+00:00") & (df.data<"2020-03-18 20:00:00+00:00") & (df.tweet.str.contains('|'.join(filtro_copom)))]\
['MLP_binary'].value_counts()


In [ ]:
df[(df.data>"2020-05-06 16:00:00+00:00") & (df.data<"2020-05-06 20:00:00+00:00") & (df.tweet.str.contains('|'.join(filtro_copom)))]\
['MLP_binary'].value_counts()


In [ ]:
df[(df.data>"2020-06-17 16:00:00+00:00") & (df.data<"2020-06-17 20:00:00+00:00") & (df.tweet.str.contains('|'.join(filtro_copom)))]\
['MLP_binary'].value_counts()

In [ ]:
df[(df.data>"2020-08-05 16:00:00+00:00") & (df.data<"2020-08-05 20:00:00+00:00") & (df.tweet.str.contains('|'.join(filtro_copom)))]\
['MLP_binary'].value_counts()